In [1]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

Using TensorFlow backend.


## Dataset
For this example, we only need one piece of training data.

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

### Read Contents of File

In [3]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# Take a look at the first 250 characters in text
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



## Encoding

In [5]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [6]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [7]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


## Creating Training Examples
Remember our task is to feed the model a sequence and have it return to us the next character. This means we need to split our text data from above into many shorter sequences that we can pass to the model as training examples. 

The training examples we will prepapre will use a *seq_length* sequence as input and a *seq_length* sequence as the output where that sequence is the original sequence shifted one letter to the right. For example:

```input: Hell | output: ello```

In [8]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1) 

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [10]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [11]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [12]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## Building the Model
Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


## Creating a Loss Function
Now we are going to create our own loss function for this problem. This is because our model will output a (64, sequence_length, 65) shaped tensor that represents the probability distribution of each character at each timestep for every sequence in the batch.

In [14]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [15]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 5.2516758e-03 -5.7410169e-03 -2.7686404e-03 ...  3.8528582e-03
    3.9430615e-03  2.9179724e-03]
  [ 5.0941580e-03 -4.2886864e-03 -2.0782088e-03 ...  8.6413026e-03
    1.5592481e-03 -2.0951182e-03]
  [ 1.3048925e-03 -7.5164484e-04 -5.8804089e-03 ... -2.5583096e-03
   -3.3797421e-03  1.0730006e-03]
  ...
  [ 4.1696918e-03 -1.1991682e-03  7.5377254e-03 ... -6.4307344e-03
   -3.3237496e-03  5.7446030e-03]
  [ 1.3672338e-03 -5.3913007e-03  3.7635290e-03 ... -4.4851499e-03
   -1.0357957e-03  4.2656506e-03]
  [-3.4615826e-03 -3.5960511e-03  4.6711080e-03 ... -3.3801701e-03
   -4.0323623e-03  5.1656095e-03]]

 [[-5.5235340e-03  3.1025455e-05  6.2452694e-03 ... -2.7626459e-03
    1.9156712e-03  5.9473477e-03]
  [-7.4729940e-04  1.5982735e-03  3.3426988e-03 ... -2.2754383e-03
    3.6895364e-03  5.0215973e-03]
  [-7.9375325e-04 -5.1808693e-03  4.9773958e-03 ...  9.9693739e-04
    7.2231106e-03  3.2539689e-03]
  ...
  [ 1.0038590e-02  1.6740793e-03  2.4718726e-03 ... -1.1085863e

In [16]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[ 0.00525168 -0.00574102 -0.00276864 ...  0.00385286  0.00394306
   0.00291797]
 [ 0.00509416 -0.00428869 -0.00207821 ...  0.0086413   0.00155925
  -0.00209512]
 [ 0.00130489 -0.00075164 -0.00588041 ... -0.00255831 -0.00337974
   0.001073  ]
 ...
 [ 0.00416969 -0.00119917  0.00753773 ... -0.00643073 -0.00332375
   0.0057446 ]
 [ 0.00136723 -0.0053913   0.00376353 ... -0.00448515 -0.0010358
   0.00426565]
 [-0.00346158 -0.00359605  0.00467111 ... -0.00338017 -0.00403236
   0.00516561]], shape=(100, 65), dtype=float32)


In [17]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

65
tf.Tensor(
[ 0.00525168 -0.00574102 -0.00276864 -0.00313823 -0.00311527 -0.00758283
  0.00162642 -0.00249862 -0.00217521 -0.00403886 -0.000759    0.00298938
 -0.00059925  0.0008098  -0.00405735 -0.00033316  0.0002782  -0.00040499
  0.00165417 -0.00067191  0.00321821 -0.0026378   0.00279826 -0.00527506
  0.00250471 -0.00043828 -0.00493765  0.00243706 -0.00318359  0.00213573
 -0.00586139  0.00318873  0.00012816 -0.00221743 -0.00562418  0.00299137
  0.00325543 -0.00250484  0.00257275 -0.00644769 -0.00352658  0.00109029
  0.00133318 -0.00300283 -0.00112588  0.00219755 -0.00065659 -0.0003406
  0.00367302 -0.00232274 -0.00151136 -0.00109861  0.00425863  0.00268207
 -0.00184465  0.0045941   0.00118531 -0.00313647 -0.00392849  0.00150481
  0.00233619 -0.00610799  0.00385286  0.00394306  0.00291797], shape=(65,), dtype=float32)


In [18]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"DOzrrTCJJdYP-ic;ilT!BRUBLAfGyBUVZprBCev$!oajSm-, W&Yke\n-&mPJFlB,E\nBmpb.XhgUfor;;dIssKTZeVX?!?W'M?OKT"

In [19]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [20]:
model.compile(optimizer='adam', loss=loss)

## Creating Checkpoints
Now we are going to setup and configure our model to save checkpoinst as it trains. This will allow us to load our model from a checkpoint and continue training it.

In [21]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## Training

In [22]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Train for 172 steps
Epoch 1/50
172/172 [==============================] - 11s 63ms/step - loss: 2.5921
Epoch 2/50
172/172 [==============================] - 10s 56ms/step - loss: 1.8855
Epoch 3/50
172/172 [==============================] - 10s 59ms/step - loss: 1.6361
Epoch 4/50
172/172 [==============================] - 10s 56ms/step - loss: 1.5039
Epoch 5/50
172/172 [==============================] - 10s 59ms/step - loss: 1.4241
Epoch 6/50
172/172 [==============================] - 10s 58ms/step - loss: 1.3675
Epoch 7/50
172/172 [==============================] - 10s 59ms/step - loss: 1.3235
Epoch 8/50
172/172 [==============================] - 10s 56ms/step - loss: 1.2869
Epoch 9/50
172/172 [==============================] - 10s 56ms/step - loss: 1.2515
Epoch 10/50
172/172 [==============================] - 10s 60ms/step - loss: 1.2178
Epoch 11/50
172/172 [==============================] - 10s 59ms/step - loss: 1.1848
Epoch 12/50
172/172 [==============================] - 10s 61ms/s

## Loading the Model

In [23]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [24]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

## Generating Text

In [25]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0) # To create a nested list which represents one batch 

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [26]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: Hello
Hellow is the point.

ISABELLA:
Sir, but needer's he.
And, Clarence, now thou art said 'Ay.'
And, liss, you are treed, you are too sorrow.

LEONTES:
O sweet Bianca, be a pape.

POMPEY:
I beseech you, sir, be merry; I do think of the
voice is a sickness is not cocketh that there will I come
And lead thy death appear, unto me well,
As I am nd you all my day.

JULIET:
After.

CAPULET:
Alas! what would I cannot do.

ROMEO:
Before it to thyself and my wedded men
And his shore purch am up, sure, knave you not
appearing me to crus so, but your mother was proclaim
False to him to a horse,
Never cannot many that bold still wash'd your heart what is
broken on?

PAULINA:
Well, poor whom I pence his summer flies;
I am inform'd that he hath observed it?
As if I love his head in holy friar,
Our army is not s
